# Equivalent Circuit Model Identification
## Estimating ECM parameters from multi-pulse HPPC data

This notebook provides example usage for estimating stationary parameters for a two RC branch Thevenin model using multi-pulse HPPC data.

### Setting up the Environment

If you don't already have PyBOP installed, check out the [installation guide](https://pybop-docs.readthedocs.io/en/latest/installation.html) first.

We begin by upgrading some dependencies and importing the necessary libraries. Let's also fix the random seed to generate consistent output during development.

In [ ]:
%pip install --upgrade pip pandas -q

import numpy as np
import pandas as pd
import pybamm

import pybop

pybop.plot.PlotlyManager().pio.renderers.default = "notebook_connected"

np.random.seed(8)  # users can remove this line

/home/nicola/GitHub/PyBOP/.nox/notebooks-overwrite/bin/python3: No module named pip


Note: you may need to restart the kernel to use updated packages.


## Setting up the model

In this example, we use the default parameter value for the "Open-circuit voltage [V]" as provided by the original PyBaMM class. The other relevant parameters for the ECM model implementation are updated as per the cell specification.

In [ ]:
# Load the parameters
parameter_values = pybamm.equivalent_circuit.Thevenin().default_parameter_values
parameter_values.update(
    {
        "Cell capacity [A.h]": 3,
        "Nominal cell capacity [A.h]": 3,
        "Upper voltage cut-off [V]": 4.25,  # extended to avoid hitting event
        "Lower voltage cut-off [V]": 2.5,
        "R0 [Ohm]": 1e-3,
        "Element-1 initial overpotential [V]": 0,
        "R1 [Ohm]": 3e-3,
        "C1 [F]": 5e2,
    }
)
# Optional arguments - only needed for two RC pairs
parameter_values.update(
    {
        "Element-2 initial overpotential [V]": 0,
        "R2 [Ohm]": 2e-3,
        "C2 [F]": 3e4,
    },
    check_already_exists=False,
)

Next we create the model with two RC elements.

In [ ]:
model = pybamm.equivalent_circuit.Thevenin(options={"number of rc elements": 2})

## Importing data

Here we will use multiple HPPC pulses from an open dataset [1].

[1] Kollmeyer, Phillip; Skells, Michael (2020), “Samsung INR21700 30T 3Ah Li-ion Battery Data”, Mendeley Data, V1, doi: 10.17632/9xyvy2njj3.1 

The initial state can be either an initial SoC or initial open-circuit voltage. In this example, we get the initial OCV by accessing the voltage data.

In [ ]:
file_loc = r"../../data/Samsung_INR21700/multipulse_hppc.xlsx"
df = pd.read_excel(file_loc, index_col=None, na_values=["NA"])
df = df.drop_duplicates(subset=["Time"], keep="first")

dataset = pybop.Dataset(
    {
        "Time [s]": df["Time"].to_numpy(),
        "Current function [A]": df["Current"].to_numpy(),
        "Voltage [V]": df["Voltage"].to_numpy(),
    }
)
parameter_values.set_initial_state(f"{df['Voltage'].to_numpy()[0]} V");

Let's take a look at the input current.

In [ ]:
pybop.plot.trajectories(
    x=df["Time"].to_numpy(),
    y=df["Current"].to_numpy(),
    title="Current vs Time",
    xaxis_title="Time / s",
    yaxis_title="Current / A",
);

## Identifying the parameters

Now we can start the PyBOP fitting process by defining the parameters for identification. The initial guess for the resistance parameter is generated from a random sample of the prior distributions. These are influenced by the `r_guess` parameter below.

In [ ]:
r0_guess = 0.005
parameters = [
    pybop.Parameter(
        "R0 [Ohm]",
        prior=pybop.Gaussian(r0_guess, r0_guess / 10),
        bounds=[0, 0.1],
    ),
    pybop.Parameter(
        "R1 [Ohm]",
        prior=pybop.Gaussian(r0_guess, r0_guess / 10),
        bounds=[0, 0.1],
    ),
    pybop.Parameter(
        "R2 [Ohm]",
        prior=pybop.Gaussian(r0_guess, r0_guess / 10),
        bounds=[0, 0.1],
    ),
    pybop.Parameter(
        "C1 [F]",
        prior=pybop.Gaussian(500, 100),
        bounds=[100, 1000],
    ),
    pybop.Parameter(
        "C2 [F]",
        prior=pybop.Gaussian(2000, 500),
        bounds=[1000, 10000],
    ),
]

Next we construct the problem builder and use it to build the optimisation problem.

In [ ]:
builder = (
    pybop.builders.Pybamm()
    .set_dataset(dataset)
    .set_simulation(model, parameter_values=parameter_values)
    .add_cost(pybop.costs.pybamm.SumSquaredError("Voltage [V]", "Voltage [V]"))
)
for param in parameters:
    builder.add_parameter(param)
problem = builder.build()

Next, we construct the optimisation class with our algorithm of choice and run it. In this case, we select the CMA-ES method as it provides global optimisation capability. For the sake of reducing the runtime of this example, we limit the maximum iterations; however, feel free to update this value.

In [ ]:
options = pybop.PintsOptions(
    sigma=[1e-3, 1e-3, 1e-3, 10, 10],
    max_unchanged_iterations=20,
    max_iterations=250,
)
optim = pybop.CMAES(problem, options=options)
results = optim.run()

## Plotting and Visualisation

Next, we use PyBOP's plotting utilities to visualise the results of the optimisation. This provides us with a visual confirmation of the optimiser's converged parameter values in the time-domain output.

In [ ]:
sim = pybamm.Simulation(model, parameter_values=results.parameter_values)
sol = sim.solve(
    t_eval=[dataset["Time [s]"][0], dataset["Time [s]"][-1]],
    t_interp=dataset["Time [s]"],
)
pybop.plot.trajectories(
    x=dataset["Time [s]"],
    y=[dataset["Voltage [V]"], sol["Voltage [V]"].data],
    trace_names=["Dataset", "Simulation"],
    xaxis_title="Time / s",
    yaxis_title="Voltage / V",
);

/home/nicola/GitHub/PyBOP/.nox/notebooks-overwrite/lib/python3.12/site-packages/pybamm/simulation.py:122: UserWarning:

The default solver changed to IDAKLUSolver after the v25.4.0. release. You can swap back to the previous default by using `pybamm.CasadiSolver()` instead.



### Convergence and Parameter Trajectories

To assess the optimisation process, we can plot the convergence of the cost function and the trajectories of the parameters:

In [ ]:
pybop.plot.convergence(optim)
pybop.plot.parameters(optim);

### Conclusion

This notebook illustrates how to perform parameter estimation for multi-pulse HPPC data, providing insights into the optimisation process through various visualisations.